<a href="https://colab.research.google.com/github/xuhu357/DataAnalysis/blob/master/ch07_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A4%80%EB%B9%84%ED%95%98%EA%B8%B0_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 겹치는 데이터 합치기

데이터를 합칠 때, merge or concat 으로 불가능한 상황이 있음.

두 데이터 셋의 색인이 일부 겹치거나 전체가 겹치는 경우가 그럼.

벡터화된 if-else 구문을 표현하는 numpy의 where 함수를 통해 자세히 알아 보자.

In [0]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np

In [0]:
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan], index=['f', 'e', 'd', 'c', 'b', 'a'])

In [0]:
b = Series(np.arange(len(a), dtype=np.float64), index=['f', 'e', 'd', 'c', 'b', 'a'])

In [0]:
b[-1] = np.nan

In [5]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [6]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [7]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

Series의 combine_first 라는 메소드는 위와 동일한 연산을 제공하며, **데이터 정렬기능**까지 제공.

In [9]:
b[:-2]

f    0.0
e    1.0
d    2.0
c    3.0
dtype: float64

In [10]:
a[2:]

d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [8]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

간단하게 말하면, 

앞에 항에 없는?? 누락된 항목은 뒤의 항의 값으로 대체한다는 말이다.

DataFrame에서 combine_first 메소드는 칼럼에 대해 같은 동작을 한다. 

그래서 이를 통해서 호출하는 객체에서 누락된 값은 인자로 넘긴 객체에 있는 값으로 채워 넣을 수 있다.

In [0]:
df1 = DataFrame({
    'a': [1., np.nan, 5., np.nan],
    'b': [np.nan, 2., np.nan, 6.],
    'c': range(2, 18, 4)
})

df2 = DataFrame({
    'a': [5., 4., np.nan, 3., 7.], 
    'b': [np.nan, 3., 4., 6., 8.]
})

In [15]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [16]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [17]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 재형성과 피벗

표 형식의 데이터를 재배치하는 다양한 기본 연산이 존재하는데, 이런 연산을 재형성 reshaping 또는 피벗 연산이라고 한다.

### 계층적 색인으로 재형성하기.

계층적 색인은 다음과 같은 DataFrame의 데이터를 재배치하는 한결같은 방식을 제공한다.

* stack: 데이터의 칼럼을 로우로 피벗 또는 회전시킨다.
* unstack: 로우를 칼럼으로 피벗시킨다.

감이 잘 오지 않으면, 일단 예제를 보면서 이해 하도록 하자.

In [0]:
data = DataFrame(np.arange(6).reshape((2, 3)), index=pd.Index(['Ohio', 'Colorado'], name='state'), 
                columns=pd.Index(['one', 'two', 'three'], name='number'))

In [19]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


stack을 사용하면, 칼럼이 로우로 피벗된다.

In [0]:
result = data.stack()

In [21]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [22]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


보통 제일 안쪽에 있는 것부터 끄집어 내는데 (stack도 마찬가지임), 레벨 이름이나, 숫자를 전달해서 끄집어 낼 단계(level)를 지정할 수 있다. 

In [23]:
result.unstack(0) # level 숫자로 끄집어 내는 경우.

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [24]:
result.unstack('state') # 이름으로 끄집어 내는 경우

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [25]:
result.unstack(level=1) # 만약 number를column으로 보내고 싶은 경우

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [26]:
result.unstack('number')

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


해당 레벨에 있는 모든 값이 하위 그룹에 속하지 않을 경우 unstack을 하게 되면, 누락된 데이터가 생길 수 있다.

In [0]:
s1 = Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])

In [0]:
s2 = Series([4, 5, 6], index=['c', 'd', 'e'])

In [0]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])

In [30]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [31]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


stack 메소드는 누락된 데이터를 자동으로 걸러내기 때문에 연산을 쉽게 원상 복구할 수 있다. 


In [32]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

nan값을 굳이 남기고 싶다면, dropna 파라미터를 False로 해주면 된다.

In [33]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

DataFrame을 unstack할때, unstack() 레벨은 결과에서 가장 낮은 단계가 된다. 


In [0]:
df = DataFrame({
    'left': result, 
    'right': result+5
}, columns=pd.Index(['left', 'right'], name='side'))

In [35]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [36]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [37]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7